In [12]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from cvxopt import matrix, solvers
merged_data = pd.read_csv('../../data/merged_data_truncated.csv')

In [13]:
merged_data.shape

(17, 8)

In [14]:
merged_data.head()

,Open,High,Low,Close,Adj Close,Volume,Sentiment1,Sentiment2
0,173.800003,177.990005,173.179993,177.490005,177.490005,57224100,0.9480,0.096352
1,176.809998,179.050003,175.800003,178.990005,178.990005,42390800,-0.2204,0.071875
2,178.100006,179.720001,177.949997,178.389999,178.389999,43698000,0.9768,0.170134
3,178.199997,179.850006,177.600006,179.800003,179.800003,47551100,0.9854,0.143628
4,180.070007,182.339996,179.039993,180.710007,180.710007,56743100,0.9934,0.106639


In [15]:
# y_open=merged_data['Open']
# y_close=merged_data['Close']
y_data=merged_data.drop(columns=['High','Low','Adj Close','Volume','Sentiment1','Sentiment2','Close'])
# print(y_data)
# y_array=y_data.values
# print(y_array)
input_vector= merged_data.drop(columns=['Open','Close','Adj Close'])
# input_array = input_vector.values
size=len(input_vector)
print(size)
train_ratio=0.6
train_size=int(size*train_ratio)
    


# #Splitting the data in training and testing data 


#Training data
X_train=input_vector[:train_size]
# print(X_train)
y_train=y_data[:train_size]
    
#Testing data

X_test=input_vector[train_size:]
y_test=y_data[train_size:]

#Conversion from pandas data frame to numpy data frame
X_train = X_train.values
X_test = X_test.values
y_train = y_train.values
y_test = y_test.values
# print(X_train)






17


In [16]:
def rbf_kernel(x1, x2, gamma='scale'):
      # If gamma is set to 'scale', calculate it based on the number of features in x1
    if gamma == 'scale':
        gamma = 1 / (2 * len(x1))
    #this is the radial basis kernel evaluation based on norm
    return np.exp(-gamma * np.linalg.norm(x1 - x2) ** 2)

In [17]:
def calculate_kernel_matrix(X, gamma='scale'):
    n_samples = X.shape[0]
    kernel_matrix = np.zeros((n_samples, n_samples))
    for i in range(n_samples):
        for j in range(n_samples):
            kernel_matrix[i, j] = rbf_kernel(X[i], X[j], gamma)
    return kernel_matrix


In [18]:
# def train_svr(X, y, C=1.0, epsilon=0.1, gamma='scale'):
#     n_samples, n_features = X.shape

#     # Calculate the kernel matrix
#     kernel_matrix = calculate_kernel_matrix(X, gamma)

#     # Quadratic programming problem (simplified)
#     P = np.outer(y, y) * kernel_matrix
#     q = -np.ones(n_samples)
#     G = np.vstack([-np.eye(n_samples), np.eye(n_samples)])
#     h = np.hstack([np.zeros(n_samples), np.ones(n_samples) * C])

#     # Solve the optimization problem (you may replace this with a more advanced solver)
#     # solution = np.linalg.solve(P, q, G, h)
#     solution = solvers.qp(P, q, G, h)

#     # Extract weights, intercept, and support vectors
#     alpha = solution[:n_samples]
#     sv_indices = np.where(alpha > 1e-5)[0]
#     support_vectors_X = X[sv_indices]
#     support_vectors_y = y[sv_indices]
#     support_vectors_alpha = alpha[sv_indices]

#     # Calculate weights and intercept
#     weights = np.sum(support_vectors_alpha * support_vectors_y[:, None] *
#                      np.array([rbf_kernel(x, support_vectors_X, gamma) for x in X.values]), axis=0)
#     intercept = np.mean(support_vectors_y - np.dot(weights, support_vectors_X.values.T))

#     return weights, intercept, {'X': support_vectors_X, 'y': support_vectors_y, 'alpha': support_vectors_alpha}


In [41]:
# # from cvxopt import matrix, solvers

# def train_svr(X, y, C=1.0, epsilon=0.1, gamma='scale'):
#     n_samples, n_features = X.shape

#     # Calculate the kernel matrix
#     kernel_matrix = calculate_kernel_matrix(X, gamma)

#     # Quadratic programming problem (simplified)
#     P = matrix(np.outer(y.flatten(), y.flatten()) * kernel_matrix)
#     q = matrix(-np.ones(n_samples))
#     G = matrix(np.vstack([-np.eye(n_samples), np.eye(n_samples)]))
#     h = matrix(np.hstack([np.zeros(n_samples), np.ones(n_samples) * C]))

#     # Solve the optimization problem using cvxopt
#     sol = solvers.qp(P, q, G, h)

#     # Extract Lagrange multipliers
#     alpha = np.array(sol['x']).flatten()

#     # Extract support vectors
#     sv_indices = np.where(alpha > 1e-5)[0]
#     support_vectors_X = X[sv_indices]
#     support_vectors_y = y[sv_indices]
#     support_vectors_alpha = alpha[sv_indices]

# # Calculate weights and intercept
#     weights = np.sum(support_vectors_alpha[:, None, None] *
#                 support_vectors_y[:, :, None] *
#                 np.array([rbf_kernel(x, support_vectors_X, gamma) for x in X]), axis=0)

#     intercept = np.mean(support_vectors_y - np.sum(weights * np.array([rbf_kernel(x, support_vectors_X, gamma) for x in X]), axis=0))

#     return weights, intercept, {'X': support_vectors_X, 'y': support_vectors_y, 'alpha': support_vectors_alpha} 


# from cvxopt import matrix, solvers
P,q,G,h=0,0,0,0

def train_svr(X, y, C=1.0, epsilon=0.1, gamma='scale'):
    global P,q,G,h
    n_samples, n_features = X.shape

    # Normalize input features if needed

    # Calculate the kernel matrix
    kernel_matrix = calculate_kernel_matrix(X, gamma)

    # Quadratic programming problem (simplified)
    P = matrix(np.outer(y.flatten(), y.flatten()) * kernel_matrix)
    q = matrix(-np.ones(n_samples))
    G = matrix(np.vstack([-np.eye(n_samples), np.eye(n_samples)]))
    h = matrix(np.hstack([np.zeros(n_samples), np.ones(n_samples) * C]))

    # Solve the optimization problem using cvxopt
    sol = solvers.qp(P, q, G, h)

    # Extract Lagrange multipliers
    alpha = np.array(sol['x']).flatten()

    # Extract support vectors
    sv_indices = np.where(alpha > 1e-5)[0]
    support_vectors_X = X[sv_indices]
    support_vectors_y = y[sv_indices]
    support_vectors_alpha = alpha[sv_indices]

    # Calculate weights and intercept
    weights = np.sum(support_vectors_alpha[:, None, None] *
                     support_vectors_y[:, :, None] *
                     np.array([rbf_kernel(x, support_vectors_X, gamma) for x in X]), axis=0)

    intercept = np.mean(support_vectors_y - np.sum(weights * np.array([rbf_kernel(x, support_vectors_X, gamma) for x in X]), axis=0))

    return weights, intercept, {'X': support_vectors_X, 'y': support_vectors_y, 'alpha': support_vectors_alpha}


In [20]:
def predict_svr(X_test, weights, intercept, support_vectors, gamma='scale'):
    
    kernel_values = np.array([rbf_kernel(x, support_vectors['X'], gamma) for x in X_test])
    print(type(weights))
    print(type(kernel_values))
    kernel_values = kernel_values.reshape(1, -1)
    weights = weights.T

    # Perform the dot product and add the intercept
    predictions = np.dot(weights, kernel_values) + intercept
    
    return predictions

In [21]:
# weights, intercept, support_vectors =train_svr(X_train, y_train, C=1.0, epsilon=0.1, gamma='scale')
# y_pred=predict_svr(X_test,weights,intercept,support_vectors)



In [22]:
def analysis(y_test,y_pred):
    # y_true = y_test  # Replace true_values with the actual values from your dataset
    # y_pred = predict_svr(X_test, weights, intercept, support_vectors, gamma)
    # print(y_pred)
    # print(y_test)
# Calculate the mean squared error (MSE)
    mse = mean_squared_error(y_test, y_pred)
    print(f'Mean Squared Error (MSE): {mse}')

# Calculate the mean absolute error (MAE)
    mae = mean_absolute_error(y_test, y_pred)
    print(f'Mean Absolute Error (MAE): {mae}')

# Calculate the R-squared (R2) score
    r2 = r2_score(y_test, y_pred)
    print(f'R-squared (R2) Score: {r2}')
    

In [33]:
weights, intercept, support_vectors =train_svr(X_train, y_train, C=1.0, epsilon=0.1, gamma='scale')
y_pred=predict_svr(X_test,weights,intercept,support_vectors)
# y_pred=y_pred[0]


     pcost       dcost       gap    pres   dres
 0: -4.0491e-08 -1.0001e+01  1e+01  4e-17  3e-16
 1: -1.4283e-04 -1.0028e-01  1e-01  2e-16  5e-16
 2: -1.5894e-04 -1.1673e-03  1e-03  2e-16  6e-17
 3: -1.5905e-04 -1.6914e-04  1e-05  2e-16  1e-16
 4: -1.5905e-04 -1.5915e-04  1e-07  2e-16  1e-16
 5: -1.5905e-04 -1.5905e-04  1e-09  2e-16  1e-16
Optimal solution found.
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [40]:
y_test[2]

array([173.05000305])

In [29]:
# y_test.shape
y_pred.shape

(10, 7)

In [34]:
analysis(y_test,y_pred)

Mean Squared Error (MSE): 45.7778908217936
Mean Absolute Error (MAE): 6.277712358747221
R-squared (R2) Score: -6.188492638766343


In [32]:
y_pred[4]

array([177.34199982, 177.34199982, 177.34199982, 177.34199982,
       177.34199982, 177.34199982, 177.34199982])

In [31]:
y_pred[1]

array([177.34199982, 177.34199982, 177.34199982, 177.34199982,
       177.34199982, 177.34199982, 177.34199982])

In [30]:
y_test.shape
y_pred.shape
y_pred[0]

array([177.34199982, 177.34199982, 177.34199982, 177.34199982,
       177.34199982, 177.34199982, 177.34199982])

In [ ]:
# import numpy as np
# from sklearn.model_selection import train_test_split

# # Assuming merged_data has columns like 'Date', 'Adj Close', 'Open', 'News_Sentiment', and other relevant features
# features = merged_data[['High', 'Low', 'Volume','Sentiment1','Sentiment2']]  # Adjust features accordingly
# # target = merged_data[['Open','Close']]  # Assuming you have a column 'Next_Day_Adj_Close'
# target = merged_data['Close']

# # Split the dataset into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# def rbf_kernel(X1, X2, gamma='auto'):
#     if gamma == 'auto':
#         # gamma = 1 / len(X1[0])
#         gamma = 1 / X1.shape[0]
#     return np.exp(-gamma * np.linalg.norm(X1 - X2) ** 2)

# def fit_svr(X, y, kernel='rbf', C=1.0, epsilon=0.1, gamma='auto'):
#     n_samples, n_features = X.shape

#     # Compute the kernel matrix
#     kernel_matrix = np.zeros((n_samples, n_samples))
#     for i in range(n_samples):
#         for j in range(n_samples):
#             if kernel == 'rbf':
#                 kernel_matrix[i, j] = rbf_kernel(X[i], X[j], gamma)

#     # Quadratic programming problem to solve the dual problem
#     P = np.outer(y, y) * kernel_matrix
#     q = -np.ones(n_samples)
#     G = np.vstack((-np.eye(n_samples), np.eye(n_samples)))
#     h = np.hstack((np.zeros(n_samples), np.ones(n_samples) * C))

#     # Solve the quadratic programming problem
#     alpha = np.linalg.solve(P, q)

#     # Extract support vectors
#     sv_indices = alpha > 1e-5
#     support_vectors = X[sv_indices]
#     support_vector_labels = y[sv_indices]

#     # Calculate weights and bias
#     weights = np.sum(alpha[sv_indices][:, np.newaxis] * support_vector_labels[:, np.newaxis] * support_vectors, axis=0)
#     bias = np.mean(support_vector_labels - np.dot(support_vectors, weights))

#     return support_vectors, support_vector_labels, weights, bias

# # def predict_svr(X, support_vectors, weights, bias, kernel='rbf', gamma='auto'):
# #     if kernel == 'rbf':
# #         kernel_values = np.array([rbf_kernel(X, sv, gamma) for sv in support_vectors])
# #     else:
# #         kernel_values = np.dot(X, support_vectors.T)

# #     return np.dot(kernel_values, weights) + bias
# def predict_svr(X, support_vectors, weights, bias, kernel='rbf', gamma='auto'):
#     if kernel == 'rbf':
#         kernel_values = np.array([rbf_kernel(X_i, sv, gamma) for X_i in X for sv in support_vectors])
#     else:
#         kernel_values = np.dot(X, support_vectors.T)

#     return np.dot(kernel_values, weights) + bias

# # Usage:
# support_vectors, sv_labels, sv_weights, sv_bias = fit_svr(X_train.values, y_train.values)
# y_pred = predict_svr(X_test.values, support_vectors, sv_weights, sv_bias)

# def analysis(y_test,y_pred):
#     # y_true = y_test  # Replace true_values with the actual values from your dataset
#     # y_pred = predict_svr(X_test, weights, intercept, support_vectors, gamma)

# # Calculate the mean squared error (MSE)
#     mse = mean_squared_error(y_test, y_pred)
#     print(f'Mean Squared Error (MSE): {mse}')

# # Calculate the mean absolute error (MAE)
#     mae = mean_absolute_error(y_test, y_pred)
#     print(f'Mean Absolute Error (MAE): {mae}')

# # Calculate the R-squared (R2) score
#     r2 = r2_score(y_test, y_pred)
#     print(f'R-squared (R2) Score: {r2}')

